<a href="https://colab.research.google.com/github/fzanart/GHDomains/blob/main/Ownership.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#Ownership
import os
import pandas as pd
from tqdm import tqdm
import json
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu
#%%


In [16]:
directory = '/content/drive/MyDrive/GHDomains/Downloaded_repos/'

In [17]:
data = []
to_rescan = []

for filename in tqdm(os.listdir(directory)):
    if not filename.startswith('.'):
        try:
            # Read the file
            aux_reader      = pd.read_json(directory+filename)
            # Correct repo name
            filename   = filename.replace(':', '/')
            filename   = filename.replace('.json', '')
            # Count contributors
            aux_dict        = Counter(aux_reader['Authors:'])
            # Save the total number of contributors
            contributors    = len(aux_dict)
            # Normalize values to calculate ownership
            factor          = 1.0/sum(aux_dict.values())
            norm_dict       = {k: v*factor for k, v in aux_dict.items()}
            ownership       = max(norm_dict, key = lambda k: norm_dict[k])
            # Save ownership
            ownership       = norm_dict[ownership]
            # Save total of major contributors
            major_contrib   = len({k:v for (k,v) in norm_dict.items() if v >= 0.05})
            # Save total of minor contributors
            minor_contrib   = contributors - major_contrib
            # Save totals
            data.extend([(filename, ownership, major_contrib, minor_contrib, contributors)])
        except:
            print('None commits found in: ', filename)
            filename   = filename.replace(':', '/')
            filename   = filename.replace('.json', '')
            to_rescan.extend([filename])

#%%
df = pd.DataFrame(data, columns =['Name', 'Ownership', 'Major contributors', 'Minor contributors', 'Total contributors'])

100%|██████████| 441/441 [00:07<00:00, 60.23it/s]


In [18]:
to_rescan

[]

In [19]:
df

,Name,Ownership,Major contributors,Minor contributors,Total contributors
0,limetext/lime,0.314312,3,56,59
1,pypa/pipenv,0.368817,3,403,406
2,deezer/spleeter,0.360587,4,14,18
3,fchollet/deep-learning-with-python-notebooks,0.666667,3,3,6
4,microsoft/Web-Dev-For-Beginners,0.265613,4,133,137
...,...,...,...,...,...
435,iview/iview-admin,0.502165,5,11,16
436,hubotio/hubot,0.240077,5,288,293
437,johnpapa/angular-styleguide,0.525528,2,192,194
438,netlify/netlify-cms,0.227797,4,490,494


In [ ]:
#TODO: Get the labels for this list of repos, to get the labels it is necessary to download the data.
1/0



In [ ]:



# len(to_rescan)


In [ ]:

# with open('contributor_rescan.txt', 'w') as f:
#     for item in to_rescan:
#         f.write('%s\n' % item)
# %%
odf = pd.read_csv('Resources/expanded_frame_final.csv')
odf = odf[['Domain', 'Name']]

join_data = pd.merge(odf, 
                      df, 
                      on ='Name', 
                      how ='inner')



# %%
odf = pd.read_csv('Resources/expanded_frame_final.csv')

odf.dropna(subset=['Readme'], inplace=True)
odf.drop(31, axis=0, inplace=True)   #removed repo Homebrew/legacy-homebrew      # Software tools
odf.drop(124, axis=0, inplace=True)  #removed repo shadowsocks/shadowsocks       # Software tools
odf.drop(237, axis=0, inplace=True)  #removed repo npm/npm                       # Software tools
odf.drop(3057, axis=0, inplace=True) #removed repo firstopinion/formatter.js     # Web libraries and frameworks
odf.drop(4488, axis=0, inplace=True) #removed repo jersey/jersey                 # Web libraries and frameworks
odf.reset_index(inplace=True, drop=True)

lista_1 = join_data['Name'].tolist()
lista_2 = odf['Name'].tolist()
list(set(lista_2) - set(lista_1))
#%%

df = join_data
df['Domain'] = df['Domain'].replace(['System software','Application software'], 'Application & System software')



domains = df['Domain'].unique()
distributions = []
medians = []
for domain in domains:
    print(domain, ':')
    data1 = df[df['Domain'] == domain]['Ownership']
    distributions.append(data1)
    medians.append(data1.median())
    for other_domain in domains[domains != domain]:
        print(other_domain)
        data2 = df[df['Domain'] == other_domain]['Ownership']
        stat, p = mannwhitneyu(data1, data2)
        print('Statistics=%.3f, p=%.3f' % (stat, p))
        # interpret
        alpha = 0.05
        if p > alpha:
            print('Same distribution (fail to reject H0)')
        else:
            print('Different distribution (reject H0)')
    print('=========================================')


# %%


for dist, domain in zip(distributions, domains):
    sns.distplot(dist,hist=False, label=domain, kde_kws={'clip': (0.0, 1.0)})
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),
          fancybox=True, shadow=True, ncol=6)
plt.tight_layout()
plt.savefig('myimage.svg', format='svg', dpi=1200)
plt.show()
# %%
